In [1]:
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load        Tokens  Owns (effective)  Host ID                               Rack 
UN  172.27.0.4  87.73 KiB   16      100.0%            75ca6567-5b44-4628-b6f9-11bfe6c14d10  rack1
UN  172.27.0.3  87.74 KiB   16      100.0%            15207ec3-aa22-4d39-8551-50d0845662ea  rack1
UN  172.27.0.2  113.16 KiB  16      100.0%            491b0306-0dd3-4ecb-9220-b276727eaf94  rack1



In [2]:
from cassandra.cluster import Cluster
cluster = Cluster(['p6-db-1', 'p6-db-2', 'p6-db-3'])
cass = cluster.connect()

In [3]:
cass.execute("drop keyspace if exists weather")

In [4]:
cass.execute("create keyspace weather with replication={'class': 'SimpleStrategy', 'replication_factor': 3};")

In [5]:
cass.execute("use weather")

In [6]:
cass.execute("""
create type weather.station_record(
    tmin INT,
    tmax INT
)
""")

In [7]:
cass.execute("""
create table weather.stations(
    id  TEXT,
    name TEXT static,
    date DATE,
    record weather.station_record,
    PRIMARY KEY (id, date)
) WITH CLUSTERING ORDER BY (date ASC)
""")

In [8]:
print(cass.execute("describe table weather.stations").one().create_statement)

CREATE TABLE weather.stations (
    id text,
    date date,
    name text static,
    record station_record,
    PRIMARY KEY (id, date)
) WITH CLUSTERING ORDER BY (date ASC)
    AND additional_write_policy = '99p'
    AND bloom_filter_fp_chance = 0.01
    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
    AND cdc = false
    AND comment = ''
    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}
    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
    AND memtable = 'default'
    AND crc_check_chance = 1.0
    AND default_time_to_live = 0
    AND extensions = {}
    AND gc_grace_seconds = 864000
    AND max_index_interval = 2048
    AND memtable_flush_period_in_ms = 0
    AND min_index_interval = 128
    AND read_repair = 'BLOCKING'
    AND speculative_retry = '99p';


In [9]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .appName("p6")
         .config('spark.jars.packages', 'com.datastax.spark:spark-cassandra-connector_2.12:3.4.0')
         .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions")
         .getOrCreate())

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9e564370-bccc-44bb-a7c6-8283bcadc888;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.4.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found com.github.stephenc.jcip#jcip-annotations;1.0-1 in central
	found com.gith

In [10]:
import pandas as pd

In [11]:
from pyspark.sql.functions import col, expr
df = spark.read.text("ghcnd-stations.txt")

In [12]:
df.toPandas()

,value
0,ACW00011604 17.1167 -61.7833 10.1 ST JO...
1,ACW00011647 17.1333 -61.7833 19.2 ST JO...
2,AE000041196 25.3330 55.5170 34.0 SHARJ...
3,AEM00041194 25.2550 55.3640 10.4 DUBAI...
4,AEM00041217 24.4330 54.6510 26.8 ABU D...
...,...
123341,ZI000067969 -21.0500 29.3670 861.0 WEST ...
123342,ZI000067975 -20.0670 30.8670 1095.0 MASVI...
123343,ZI000067977 -21.0170 31.5830 430.0 BUFFA...
123344,ZI000067983 -20.2000 32.6160 1132.0 CHIPI...


In [13]:
from pyspark.sql.functions import col, expr
df = df.select(
    expr("substring(value, 0, 11)").alias("id"),
    expr("substring(value, 39, 2)").alias("state"),
    expr("substring(value, 42, 30)").alias("name"),
)
df.toPandas()

,id,state,name
0,ACW00011604,,ST JOHNS COOLIDGE FLD
1,ACW00011647,,ST JOHNS
2,AE000041196,,SHARJAH INTER. AIRP
3,AEM00041194,,DUBAI INTL
4,AEM00041217,,ABU DHABI INTL
...,...,...,...
123341,ZI000067969,,WEST NICHOLSON
123342,ZI000067975,,MASVINGO
123343,ZI000067977,,BUFFALO RANGE
123344,ZI000067983,,CHIPINGE


In [14]:
df = df.where(df.state == 'WI')
df

DataFrame[id: string, state: string, name: string]

In [15]:
data = df.collect()

In [16]:
insert_statement = cass.prepare("""
INSERT INTO weather.stations (id, name)
VALUES (?, ?)
""")

for row in data:
    cass.execute(insert_statement, (row.id, row.name))

In [17]:
#q1
count_stations = cass.execute("""
SELECT COUNT(*)
FROM weather.stations
""")
cass.execute("describe table weather.stations").one()[3]

"CREATE TABLE weather.stations (\n    id text,\n    date date,\n    name text static,\n    record station_record,\n    PRIMARY KEY (id, date)\n) WITH CLUSTERING ORDER BY (date ASC)\n    AND additional_write_policy = '99p'\n    AND bloom_filter_fp_chance = 0.01\n    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}\n    AND cdc = false\n    AND comment = ''\n    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}\n    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}\n    AND memtable = 'default'\n    AND crc_check_chance = 1.0\n    AND default_time_to_live = 0\n    AND extensions = {}\n    AND gc_grace_seconds = 864000\n    AND max_index_interval = 2048\n    AND memtable_flush_period_in_ms = 0\n    AND min_index_interval = 128\n    AND read_repair = 'BLOCKING'\n    AND speculative_retry = '99p';"

In [18]:
#q2
stations = cass.execute("""
SELECT name
FROM weather.stations
WHERE id = 'USW00014837'
""")
stations.one()[0]

'MADISON DANE CO RGNL AP       '

In [19]:
#q3
stations = cass.execute("""
SELECT token(id)
FROM weather.stations
WHERE id = 'USC00470273'
""")
stations.one()[0]

-9014250178872933741

In [20]:
#q4 @@@@@@@@@@@@@@@@@@@Handle the case where the ring "wraps around" 
stations = cass.execute("""
SELECT token(id)
FROM weather.stations
WHERE token(id) > token('USC00470273')
""")
stations.one()[0]

-9008888552508841246

In [21]:
stations = cass.execute("""
SELECT token(id)
FROM weather.stations
WHERE id = 'USC00470273'
""")
stations

In [22]:
pd.DataFrame(cass.execute("select record.tmax from weather.stations"))

,record_tmax
0,None
1,None
2,None
3,None
4,None
...,...
1308,None
1309,None
1310,None
1311,None


In [23]:
# if not
# !unzip records.zip

In [24]:
spark = SparkSession.builder.appName("p6").getOrCreate()
record_df = spark.read.parquet('records.parquet')
record_df.toPandas()

,station,date,element,value
0,USW00014898,20220101,TMAX,-32.0
1,USW00014898,20220102,TMAX,-77.0
2,USW00014898,20220103,TMAX,-60.0
3,USW00014898,20220104,TMAX,0.0
4,USW00014898,20220105,TMAX,-16.0
...,...,...,...,...
19629,USR0000WDDG,20221227,TAVG,-114.0
19630,USR0000WDDG,20221228,TAVG,2.0
19631,USR0000WDDG,20221229,TAVG,72.0
19632,USR0000WDDG,20221230,TAVG,-31.0


In [25]:
type(record_df)

pyspark.sql.dataframe.DataFrame

In [26]:
record_df.rdd.take(10)

[Row(station='USW00014898', date='20220101', element='TMAX', value=-32.0),
 Row(station='USW00014898', date='20220102', element='TMAX', value=-77.0),
 Row(station='USW00014898', date='20220103', element='TMAX', value=-60.0),
 Row(station='USW00014898', date='20220104', element='TMAX', value=0.0),
 Row(station='USW00014898', date='20220105', element='TMAX', value=-16.0),
 Row(station='USW00014898', date='20220106', element='TMAX', value=-71.0),
 Row(station='USW00014898', date='20220107', element='TMAX', value=-71.0),
 Row(station='USW00014898', date='20220108', element='TMAX', value=-32.0),
 Row(station='USW00014898', date='20220109', element='TMAX', value=-27.0),
 Row(station='USW00014898', date='20220110', element='TMAX', value=-149.0)]

In [27]:
records_df = record_df.rdd.filter(lambda x: "TMAX" or "TMIN" in x.element)
# records_df = record_df.rdd.groupBy(("station","date"))       
records_df.take(10)

[Row(station='USW00014898', date='20220101', element='TMAX', value=-32.0),
 Row(station='USW00014898', date='20220102', element='TMAX', value=-77.0),
 Row(station='USW00014898', date='20220103', element='TMAX', value=-60.0),
 Row(station='USW00014898', date='20220104', element='TMAX', value=0.0),
 Row(station='USW00014898', date='20220105', element='TMAX', value=-16.0),
 Row(station='USW00014898', date='20220106', element='TMAX', value=-71.0),
 Row(station='USW00014898', date='20220107', element='TMAX', value=-71.0),
 Row(station='USW00014898', date='20220108', element='TMAX', value=-32.0),
 Row(station='USW00014898', date='20220109', element='TMAX', value=-27.0),
 Row(station='USW00014898', date='20220110', element='TMAX', value=-149.0)]

In [28]:
from pyspark.sql.functions import to_date
# records_df = record_df.rdd.filter(lambda x: "TMAX" or "TMIN" in x.element)
df_transformed = (record_df
                  .filter(record_df.element.isin(["TMIN", "TMAX"]))
                  .groupBy("station", "date")
                  .pivot("element")
                  .agg({'value': 'first'})
                  .withColumnRenamed("TMAX", "tmax")
                  .withColumnRenamed("TMIN", "tmin")
                  .withColumn("date", to_date(col("date"), "yyyyMMdd"))
                 )
records = df_transformed.collect()

In [29]:
import grpc
import station_pb2_grpc, station_pb2

channel = grpc.insecure_channel("localhost:5440")
stub = station_pb2_grpc.StationStub(channel)

for record in records:
    # print(type(record.station),type(record.date), type(record.tmin))
    response = stub.RecordTemps(station_pb2.RecordTempsRequest(
        station = record.station,
        date = str(record.date),
        tmin = int(record.tmin),
        tmax = int(record.tmax)
    ))


In [30]:
#q5
response = stub.StationMax(station_pb2.StationMaxRequest(station='USW00014837'))
response.tmax

356

In [31]:
df = (spark.read.format("org.apache.spark.sql.cassandra")
.option("spark.cassandra.connection.host", "p6-db-1,p6-db-2,p6-db-3")
.option("keyspace", "weather")
.option("table", "stations")
.load())

df.createOrReplaceTempView("stations")

In [32]:
#q6
spark.catalog.listTables()

[Table(name='stations', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [42]:
#q7
avg_diff_df = spark.sql( """
SELECT id, AVG(record.tmax - record.tmin) as avg_diff
FROM stations
GROUP BY id
HAVING avg_diff is not null
""").toPandas()
result = {}
for i, row in avg_diff_df.iterrows():
    result[row['id']] = row['avg_diff']
dict(result)

{'gas_station': 22.0,
 'USW00014898': 102.93698630136986,
 'USR0000WDDG': 102.06849315068493,
 'USW00014837': 105.62739726027397,
 'USW00014839': 89.6986301369863}

In [38]:
#q8
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load        Tokens  Owns (effective)  Host ID                               Rack 
UN  172.27.0.4  171.72 KiB  16      100.0%            75ca6567-5b44-4628-b6f9-11bfe6c14d10  rack1
UN  172.27.0.3  167.53 KiB  16      100.0%            15207ec3-aa22-4d39-8551-50d0845662ea  rack1
UN  172.27.0.2  190.6 KiB   16      100.0%            491b0306-0dd3-4ecb-9220-b276727eaf94  rack1



In [39]:
#q9
response = stub.StationMax(station_pb2.StationMaxRequest(
    station = "USW00014837"
))
response.error

''

In [40]:
#q10
response = stub.RecordTemps(station_pb2.RecordTempsRequest(
    station = "gas_station",
    date = '2023-11-22',
    tmin = 12,
    tmax = 34
))
response.error

''